In [4]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")
mapping={"settings":{"number_of_shards": 1,"number_of_replicas": 0},
"mappings": {"dynamic": "true","_source": {"enabled": "true"},
"properties": {
"title": {
"type": "search_as_you_type"
},
    "date":
    {
        "type": "date"
    },
"category":
    {
        "type":"keyword"
    }

}}}
es.indices.create(
    index="autocomplete",
    body=mapping,
    ignore=400 # ignore 400 already exists code
)

C:\Users\ivnag\AppData\Local\Temp/ipykernel_19212/4199784792.py:19: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(
C:\Users\ivnag\AppData\Local\Temp/ipykernel_19212/4199784792.py:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'autocomplete'})

In [3]:
es.indices.delete(index='autocomplete', ignore=[400, 404])

C:\Users\ivnag\AppData\Local\Temp/ipykernel_19212/1038114514.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='autocomplete', ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [6]:
data=list(i.replace("\n","") for i in open("news-dump_2020_1.txt",encoding='utf-8'))
i=0
l=len(data)
res=[]
while i<l:
    
    if (len(data[i])!=11) or (data[i].count("/")!=3):
        i+=1
    else:
        date = data[i]
        zz=data[i+1].index("$")
        category, title = data[i+1][:zz],data[i+1][zz+1:]
        i+=2
        article=""
        while i<l and (not((len(data[i])==11) and (data[i].count("/")==3))):
            article+=data[i]+" "
            i+=1
        res.append((date,category,title))
    
    #if i>100:break

In [6]:
import pandas as pd
data=pd.read_csv("news-filter-data.csv")
import pickle
fff=open("filter-content-list",'rb')
zzz=pickle.load(fff)
fff.close()
from ast import literal_eval
res=[]
for i in range(len(data)):
    res.append(tuple(data[j][i] if j=="date" else " ".join(literal_eval(data[j][i])) for j in ("date","category","headline")))
#res=[(a,b,c) for a,b,c in res if b in ('frontpag','sport','busi','nation','intern')]
mapping={'frontpag':'frontpage','sport':'sport','busi':'business','nation':'national','intern':'international'}

new_res=[]
cc=0
for a,b,c in res:
    if b in mapping:
        ind=zzz[cc].index("\n")
        titl = zzz[cc][:ind]
        doccc = zzz[cc][ind+1:]
        
        
        b=mapping[b]
        new_res.append((a,b,titl))
    cc+=1
print(len(new_res),len(zzz))
res=new_res

111877 111887


In [9]:
res[2]

('2012/01/01/', 'frontpage', "“Government won't accept all amendments”")

In [10]:
import numpy as np

indd=1
for date,category,title in res:
    date=date.replace("/","-")[:-1]
    #doc=get_embedding(doc)
    es.index(index="autocomplete" , id=indd, document={"title":title,"date":date,"category":category})
    print(indd,end='\r')
    indd+=1

In [46]:
r=''
for ii in "how are you doing":
    r+=ii
    queryy = {


        "bool":
         {
               "must":[
             { "multi_match": {
               "query": r,
               "type": "bool_prefix",
               "fields": [
                   "title",
                   "title._2gram",
                   "title._3gram"
               ]
           }}],
             "filter":
             [
                 {
                     "terms":{
                         "category":["tp-sport","tp-business","tp-national","tp-international"]
                     }

                 },
                 {
                     "range":{
                         "date":{"gte":"2020-01-01",
                         "lte":"2020-06-01"}
                     }

                 }

             ]

         }   

        }



    for i in es.search(index="autocomplete",body={'size': 10,'query': queryy})['hits']['hits']:
        print(i['_source']['title'])
    print("------------------------")

Rawat to head Military Affairs Dept.
‘Iran will be held fully responsible’
Thampi seems to be back at his best
Congress slams rail fare hike
Gehlot asks Harsh Vardhan to visit Kota hospital
Relief for Netanyahu as SC refuses to rule on his eligibility
Nifty closes at a record high amid global rally
A battle of the heavyweights
Hyderabad pacers restrict Kerala
171 hospitals de-listed from PM’s health scheme after fraud
------------------------
Gehlot asks Harsh Vardhan to visit Kota hospital
171 hospitals de-listed from PM’s health scheme after fraud
Home advantage for RKFC
Dozens held after protest in Hong Kong
Beware these home loan myths
Saini and Shardul prove too hot for Sri Lanka
SBI to extend completion guarantee for home projects
Men in Blue prove too hot for Sri Lanka, take a 1-0 lead
Yet another home series victory beckons India
Chennaiyin cruises home
------------------------
‘It’s unclear how Coronavirus is transmitted’
How long until news if Olympics will open?
How it came 

C:\Users\ivnag\AppData\Local\Temp/ipykernel_14628/4159854638.py:43: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  for i in es.search(index="autocomplete",body={'size': 10,'query': queryy})['hits']['hits']:


{'took': 40, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 44, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'autocomplete', '_id': '21', '_score': 1.0, '_source': {'title': 'Thampi seems to be back at his best', 'date': '2020-01-01', 'category': 'tp-sport'}}, {'_index': 'autocomplete', '_id': '109', '_score': 1.0, '_source': {'title': 'Bumrah’s return highlights India’s first venture in the new year', 'date': '2020-01-05', 'category': 'tp-sport'}}, {'_index': 'autocomplete', '_id': '229', '_score': 1.0, '_source': {'title': 'Chennai City stuns Gokulam in high-scoring thriller', 'date': '2020-01-10', 'category': 'tp-sport'}}, {'_index': 'autocomplete', '_id': '433', '_score': 1.0, '_source': {'title': 'Vidarbha hits back early to cut short Delhi’s joy', 'date': '2020-01-20', 'category': 'tp-sport'}}, {'_index': 'autocomplete', '_id': '457', '_score': 1.0, '_source': {'title': 'Rahul has sealed his spot as